In [ ]:
import numpy as np
import pandas as pd
# we can drop the Type column because it is always House
df = pd.read_csv(filepath_or_buffer='/kaggle/input/karachi-houses/homes.csv').drop(columns='Type')
df.head()

In [ ]:
df.info()

In [ ]:
from plotly.express import histogram
def get_area_amount(arg):
    if isinstance(arg, float): # nans
        return arg
    if arg == 'Area': # special case
        return np.nan
    return int(arg.split()[0].replace(',', ''))
df['area_amount'] = df['Area'].apply(func=get_area_amount)
histogram(data_frame=df, x='area_amount', log_y=True)

We expect our prices to be a function of the area, neighborhood, and room details; we need to get the area as an integer, and we need to deal with some noise in the data. We use a log scale here to spread out our cluster near zero. This is an exploratory histogram and probably not appropriate for the general public. 

In [ ]:
df['area_amount'].isna().sum() / len(df)

2.3% of our area data is useless; this is probably not bad.

In [ ]:
def get_area_units(arg):
    if isinstance(arg, float):
        return arg
    pieces = arg.split()
    if len(pieces) == 1:
        return np.nan
    return ' '.join(pieces[1:])
df['area_units'] = df['Area'].apply(get_area_units)
df['area_units'].value_counts(dropna=False)

All of our area is in the same units, so we don't need to do any conversions. And again about 2% of our area data is unusable.

In [ ]:
histogram(data_frame=df, x='Baths', log_y=True)

This is kind of a mess, so we will need to introduce something we can use. We assume '-' means zero instead of unknown.

In [ ]:
def get_baths(arg):
    if isinstance(arg, float):
        return arg
    if str(arg).isdigit():
        return int(arg)
    if arg == '-':
        return 0
    return np.nan
df['baths'] = df['Baths'].apply(get_baths)
histogram(data_frame=df, x='baths')    

In [ ]:
df['baths'].isna().sum()

In [ ]:
from plotly.express import bar
for column in ['Purpose',]:
    bar(data_frame=df[column].value_counts().to_frame().reset_index(), x=column, y='count').show()

We have some noise in our Purpose column we will need to clean up.

In [ ]:
df['for_sale'] = df['Purpose'].apply(func=lambda x: x == 'For Sale')
df['for_sale'].value_counts(dropna=False)

A little over 1% of our Purpose data is bad. Unfortunately because our Purpose data is essentially the same in every case it will not tell us anything that will help our analysis.